In [1]:
import torch
from flask import Flask, jsonify, request
import os
import os.path
import json
import sys
import pandas as pd
import numpy as np
from operator import itemgetter
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns
import time
from itertools import combinations
import time
from numpy import load ,save
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

app = Flask(__name__)
trX = load('data.npy')
W=torch.load('tensor_W.pt')
hb=torch.load('tensor_hb.pt')
vb=torch.load('tensor_vb.pt')
visibleUnits=3273
customer_df = pd.read_csv('customerdata.csv',low_memory=False)
rating=pd.read_csv('rating.csv')
customer_df=pd.merge(customer_df,rating,on=['status','COMPLAINT_REASON'],how='left')
items_df = customer_df[["product_item_id", "PRODUCT_NAME", "CATEGORY"]]
items_df = items_df.drop_duplicates()
items_df = items_df.reset_index(drop = True)
items_df = pd.DataFrame(items_df.groupby('product_item_id')["PRODUCT_NAME"].apply(list)).reset_index()
purchase_df=customer_df.groupby(['CUSTOMER_ID','product_item_id'])['RATING_PRODUCT'].mean().reset_index(name='RATING')
purchase_df['RATING'] = 1
user_item_df =  pd.pivot_table(purchase_df,index='CUSTOMER_ID',columns=['product_item_id'],values='RATING',aggfunc='mean')
@app.route('/sampleapi/<string:typeofscript>', methods = ['GET','POST'])
def disp(typeofscript):
   
    if typeofscript=='printcust':
        start = time.time()
        user_id = request.args.get('mock_user_id')
        user_id=int(user_id)
        mock_user_id = user_id
        # customer_df[(CUSTOMER_ID[mock_user_id]]
        # customer_df.loc[mock_user_id]
        customer_df.loc[customer_df['CUSTOMER_ID'] == mock_user_id].groupby(['CUSTOMER_ID','PRODUCT_NAME']).size().reset_index(name='RATING')

        #Selecting the input user
       
       
        inputUser = trX[mock_user_id-1].reshape(1, -1)

        inputUser = tf.convert_to_tensor(trX[mock_user_id-1],"float32")
       
        v0 = inputUser

        print(v0)
        v0.shape
       
        v0test = tf.zeros([visibleUnits], tf.float32)
        v0test.shape

        #Feeding in the user and reconstructing the input

        hh0 = tf.nn.sigmoid(tf.matmul([v0], W) + hb)

        vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(W)) + vb)

        rec = vv1

        tf.maximum(rec,1)
        for i in vv1:
            print(i)

        scored_items_df_mock = items_df[items_df['product_item_id'].isin(user_item_df.columns)]
        scored_items_df_mock = scored_items_df_mock.assign(RecommendationScore = rec[0])
        scored_items_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)

        items_df_mock = purchase_df[purchase_df['CUSTOMER_ID'] == mock_user_id]
        items_df_mock.head()

        #Merging movies_df with ratings_df by MovieID
        merged_df_mock = scored_items_df_mock.merge(items_df_mock, on='product_item_id', how='outer').drop(columns=['CUSTOMER_ID', 'RATING'])

        final=merged_df_mock.sort_values(["RecommendationScore"], ascending=False).head(20)
       
        print(time.time(),start)
        return final.to_json(orient='records')
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(3273,), dtype=float32)


127.0.0.1 - - [13/May/2021 15:18:31] "GET /sampleapi/printcust?mock_user_id=33 HTTP/1.1" 200 -


tf.Tensor(
[9.6438953e-04 5.4415985e-04 1.3479238e-03 ... 1.0740903e-05 7.5675862e-06
 5.3280601e-06], shape=(3273,), dtype=float32)
1620899311.6006792 1620899311.1307461
tf.Tensor([0. 1. 0. ... 0. 0. 0.], shape=(3273,), dtype=float32)


...

127.0.0.1 - - [13/May/2021 16:44:47] "GET /sampleapi/printcust?mock_user_id=22 HTTP/1.1" 200 -


tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(3273,), dtype=float32)
tf.Tensor(
[7.8420686e-05 1.3448541e-04 1.2065125e-04 ... 1.6954917e-05 9.0883668e-06
 5.3559165e-06], shape=(3273,), dtype=float32)
1620904487.3260636 1620904487.305106


127.0.0.1 - - [13/May/2021 16:44:50] "GET /sampleapi/printcust?mock_user_id=33 HTTP/1.1" 200 -


tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(3273,), dtype=float32)
tf.Tensor(
[9.6438953e-04 5.4415985e-04 1.3479238e-03 ... 1.0740903e-05 7.5675862e-06
 5.3280601e-06], shape=(3273,), dtype=float32)
1620904490.0305495 1620904490.0085747


127.0.0.1 - - [13/May/2021 17:05:19] "GET /Chitale%20Full%20Cream%20Milk-1L HTTP/1.1" 404 -

KeyboardInterrupt

